In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
os.environ["KERAS_BACKEND"]="jax" 
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

In [2]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2S,preprocess_input
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger

In [3]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [4]:
x_train=tf.image.resize(x_train,(64,64)).numpy()
x_test=tf.image.resize(x_test,(64,64)).numpy()

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,random_state=42,test_size=0.25,stratify=y_train)

In [6]:
datagen=ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [7]:
base_model=EfficientNetV2S(
    weights='imagenet',
    include_top=False, 
    input_shape=(64,64,3) 
)

82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
base_model.trainable=False

In [9]:
model=models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(512,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(256,activation='silu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(100,activation='softmax') 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-s (Functional)        │ (None, 2, 2, 1280)          │      20,331,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,311,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,332,100 (85.19 MB)

 Trainable params: 1,997,156 (7.62 MB)

 Non-trainable params: 20,334,944 (77.57 MB)

In [10]:
early_stopper=EarlyStopping(restore_best_weights=True,monitor="val_loss",patience=5)

In [11]:
checkpoint=ModelCheckpoint('best_model.keras',monitor='val_accuracy',save_best_only=True)

In [12]:
csv_logger=CSVLogger('training_log.csv',append=True)

In [13]:
import time
start_time=time.time()

In [14]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='sparse_categorical_crossentropy',metrics=["accuracy"],jit_compile=True)

In [15]:
history=model.fit(datagen.flow(x_train,y_train),epochs=20,batch_size=32,callbacks=[early_stopper],validation_data=[x_val,y_val],verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 119s 91ms/step - accuracy: 0.0467 - loss: 4.8515 - val_accuracy: 0.3191 - val_loss: 2.8149
Epoch 2/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 81s 61ms/step - accuracy: 0.1813 - loss: 3.5207 - val_accuracy: 0.3920 - val_loss: 2.3977
Epoch 3/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 71s 61ms/step - accuracy: 0.2374 - loss: 3.1082 - val_accuracy: 0.4249 - val_loss: 2.2079
Epoch 4/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 72s 61ms/step - accuracy: 0.2782 - loss: 2.9110 - val_accuracy: 0.4430 - val_loss: 2.1024
Epoch 5/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 71s 61ms/step - accuracy: 0.2935 - loss: 2.7827 - val_accuracy: 0.4582 - val_loss: 2.0320
Epoch 6/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 71s 61ms/step - accuracy: 0.3169 - loss: 2.6736 - val_accuracy: 0.4666 - val_loss: 1.9832
Epoch 7/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 70s 60ms/step - accuracy: 0.3300 - loss: 2.6175 - val_accuracy: 0.4740 - val_loss: 1.9475
Epoch 8/20
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 74s 63ms/step - accuracy: 0.3399 

In [16]:
head_training_time=time.time()-start_time

In [17]:
base_model.trainable=True

In [18]:
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],jit_compile=True)

In [19]:
start_fine_tune=time.time()

In [20]:
fine_tune_history=model.fit(
    datagen.flow(x_train,y_train,batch_size=32),
    validation_data=(x_val,y_val),
    epochs=10,
    callbacks=[early_stopper,csv_logger],
    verbose=1
)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 259s 182ms/step - accuracy: 0.1300 - loss: 4.2259 - val_accuracy: 0.2711 - val_loss: 3.0131
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 148s 98ms/step - accuracy: 0.2303 - loss: 3.2585 - val_accuracy: 0.3734 - val_loss: 2.4386
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 116s 99ms/step - accuracy: 0.2906 - loss: 2.8500 - val_accuracy: 0.4318 - val_loss: 2.1363
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 116s 99ms/step - accuracy: 0.3377 - loss: 2.5910 - val_accuracy: 0.4642 - val_loss: 1.9608
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 115s 98ms/step - accuracy: 0.3771 - loss: 2.3768 - val_accuracy: 0.4964 - val_loss: 1.8344


In [21]:
fine_tune_time=time.time()-start_fine_tune

In [22]:
test_loss, test_accuracy=model.evaluate(x_test,y_test,verbose=0)
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")
print(f"Total training time: {head_training_time + fine_tune_time:.2f} seconds")


Test Accuracy: 27.10%
Total training time: 2467.66 seconds


In [23]:
val_loss,val_accuracy=model.evaluate(x_val,y_val)
print(f"Validation accuracy: {val_accuracy * 100:.4f}%")

391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.2697 - loss: 3.0099
Validation accuracy: 27.1120%


In [24]:
import numpy as np
y_pred=model.predict(x_test)
y_pred_class=np.argmax(y_pred,axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step


In [25]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report


accuracy=accuracy_score(y_test,y_pred_class)
precision=precision_score(y_test,y_pred_class,average='weighted')
recall=recall_score(y_test,y_pred_class,average='weighted')
f1=f1_score(y_test,y_pred_class,average='weighted')


print("\nPerformance Metrics Breakdown for EfficientNetV2S:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


Performance Metrics Breakdown for EfficientNetV2S:
Accuracy: 0.2710
Precision: 0.2646
Recall: 0.2710
F1-Score: 0.2622


In [27]:
import pandas as pd
results=pd.DataFrame({
    "Model": ["EfficientNetV2S"],
    "Test Accuracy": [test_accuracy],
    "Validation Accuracy":[val_accuracy],
    "Test Loss": [test_loss],
    "Head Training Time (s)": [head_training_time],
    "Fine-tuning Time (s)": [fine_tune_time],
    "Total Params": [model.count_params()]
})

In [28]:
print(results)

             Model  Test Accuracy  Validation Accuracy  Test Loss  \
0  EfficientNetV2S          0.271              0.27112   2.986425   

   Head Training Time (s)  Fine-tuning Time (s)  Total Params  
0             1492.805721            974.854625      22332100  


In [29]:
results.to_csv("EfficientNetV2S.csv",index=False)
print("\nResults saved to EfficientNetV2S.csv")


Results saved to EfficientNetV2S.csv
